# Übung

In [1]:
!pip install sklearn

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Pakete importieren

In [3]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import pickle
from sklearn.model_selection import train_test_split
print('Pakete importiert.')

Packete importiert.


## Numerische Daten

### Task 1: Einlesen der Datensets

Lesen Sie die gespeicherten Datensets aus der pickle-Datei '../output/bikebuyers/datasets.pkl' aus und geben Sie die ersten fünf Zeilen der Merkmale im Trainingsdatenset (X_train) aus.

In [4]:
# Hier den Code eingeben.

Geben Sie die ersten fünf Zeilen der Zielgrößen im Trainingsdatenset (y_train) aus.

In [5]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 1

  ```{code-block} python
  # Erste Code-Zelle
  with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:      
    datasets = pickle.load(handle)
        
  datasets['X_train'].head()
    
  # Zweite Code-Zelle
  datasets['y_train'].head()
  ```
````

### Task 2: Ausreißer erkennen

Ermitteln Sie mit der IQR-Methode die Ausreißer, ersetzen diese mit dem NaN-Wert (np.nan) und geben Sie die Anzahl der Ausreißer pro Merkmal aus. Verwenden Sie den Faktor 1.5 bei der IQR-Methode.

In [6]:
# Hier den Code eingeben.

Ersetzen Sie die NaN-Werte mit dem Mittelwert.

In [7]:
# Hier den Code eingeben.

````{Tip}
Schritte zur Lösung:
* Eine Variable factor erstellen und mit dem Wert 1.5 belegen.
* Mit Hilfe der [quantile()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html)-Methode das 25%-Quantil bestimmen und in einer Variable q1 speichern.
* Mit Hilfe der [quantile()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html)-Methode das 75%-Quantil bestimmen und in einer Variable q3 speichern.
* Die Differenz von q3 und q1 berechnen und in einer Variable namens iqr speichern.
* Die Untere Grenze berechnen durch die Differenz von q1 und dem Faktor multipliziert mit iqr.
* Alle Werte die außerhalb des Bereichs liegen (definierte Grenzen) mit dem NaN-Wert belegen.
* Mit Hilfe der Methode [isna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) alle NaN-Werte identifizieren und mit Hilfe der Methode sum() die Anzahl pro Merkmal ausgeben.
* Die Methode [fillna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) hilft bei dem Ersetzen der NaN-Werte.
* Die [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.mean.html)-Methode ermittelt Mittelwerte eines DataFrames.
````

````{Dropdown} Lösung Task 2

  ```{code-block} python
  # Erste Code-Zelle
  X_ = pd.DataFrame(datasets['X_train'])
  factor = 1.5
  q1 = X_.quantile(0.25)
  q3 = X_.quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - (factor * iqr)
  upper_bound = q3 + (factor * iqr)
  X_[((X_ < lower_bound) | (X_ > upper_bound))] = np.nan
  X_.isna().sum()

  # Zweite‚ Code-Zelle
  X_.fillna(X_.mean(), inplace=True)
  ```
````

### Task 3: Outlier Remover Transformer erstellen

Erstellen Sie eine Klasse "OutlierRemoverExtended", welche das Transformer-Interface von Scikit Learn abbildet und von den Klassen BaseEstimator und TranformerMixin ableitet. Speichern Sie die Klasse in einer Datei namens "transformer_extended.py" mit Hilfe des [%%writefile](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) Befehls. Bei der Instanziierung der Klasse sollen zwei Parameter gesetzt werden können:
* factor, Default-Wert 1.5
* strategy, Default-Wert 'median'

Die fit()-Methode soll zwei numpy-Arrays als Parameter (X und y), keine Funktion enthalten und nur die Instanz selbst zurückgeben. Die Transformer-Methode soll die gleichen Parameter wie die fit()-Methode erhalten, Ausreißer mit der IQR-Methode unter Verwendung des factor-Paramters erkennen und mit dem Mittelwert oder dem Median ersetzen. Welche Art zum Einsatz kommt, wird mit dem Paramter strategy bestimmt. Valide Werte sind 'median' und 'mean'.

In [8]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 3

  ```{code-block} python
  %%writefile transformer_extended.py
  
  import pandas as pd
  import numpy as np
  from sklearn.base import BaseEstimator, TransformerMixin
  
  class OutlierRemoverExtended(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5, strategy='median'):
      self.factor = factor
      self.strategy = strategy

    def fit(self, X, y=None):
      return self

    def transform(self, X, y=None):
      X_ = pd.DataFrame(X)
      q1 = X_.quantile(0.25)
      q3 = X_.quantile(0.75)
      iqr = q3 - q1
      lower_bound = q1 - (self.factor * iqr)
      upper_bound = q3 + (self.factor * iqr)
      X_[((X_ < lower_bound) | (X_ > upper_bound))] = np.nan

      if self.strategy == 'median':
        X_.fillna(X_.median(), inplace=True)
      elif self.strategy == 'mean':
        X_.fillna(X_.mean(), inplace=True)
      else:
        raise ValueError('Invalid value for strategy paramter. Valid values are median or mean.')

      return X_.values
  ```
````

### Task 4: Transformer anwenden

* Erstellen Sie eine Instanz der Klasse OutlierRemoverExtended.
* Wenden Sie die fit_transform()-Methode auf das Trainingsdatenset an und speichern das Ergebnis in einer Variable.
* Erstellen Sie ein Pandas DataFrame, übergeben sie dabei die transformierten Werte und geben das Ergebnis aus.

In [9]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 4

  ```{code-block} python
  from transformer_extended import OutlierRemoverExtended
  
  outlier_remover = OutlierRemoverExtended()
  res = outlier_remover.fit_transform(datasets['X_train'])
  pd.DataFrame(res)
  ```
````

### Task 5: Pipeline für numerische Daten erstellen

Erstellen Sie eine Pipeline mit Hilfe der [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)-Klasse von Scikit Learn. Schritte der Pipeline:
1. Der OutlierRemoverExtended-Transformer aus Task 3  
2. [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) von Scikit Learn

Speichern Sie die Pipeline in einer Variable namens pipeline_numerical.

In [10]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 5

  ```{code-block} python
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import StandardScaler
  
  pipeline_numerical = Pipeline(steps=[
    ('outlier_remover', OutlierRemoverExtended()),
    ('scaler', StandardScaler())
  ])
  ```
````

## Kategorische Daten

### Task 6: Pipeline für kategorische Daten erstellen

Erstellen Sie eine Pipeline mit Hilfe der [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)-Klasse von Scikit Learn. Schritte der Pipeline:
1. [OneHot-Encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) von Scikit Learn  

Speichern Sie die Pipeline in einer Variable namens pipeline_categorical.

In [11]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 6

  ```{code-block} python
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import OneHotEncoder
  
  pipeline_categorical = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])
  ```
````

## Transformationen koordinieren

### Task 7: Pipelines zusammenfügen und Merkmale zuweisen

* Bezeichnungen der numerischen Features in einer Liste mit Elementen vom Typ String namens 'features_numerical' speichern.
* Bezeichnungen der kategorischen Features in einer Liste mit Elmenten vom Typ String namens 'features_categorical' speichern.
* Eine Instanz des [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) aus Scikit Learn erstellen. Dabei dem Parameter transformers die Tuples 
    * ('num', pipeline_numerical, features_numerical)
    * ('cat', pipeline_categorical, features_categorical)  
übergeben

In [12]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 7

  ```{code-block} python
  from sklearn.compose import ColumnTransformer

  features_numerical = ['Income', 'Age', 'Cars', 'Children']
  features_categorical = [
    'Marital Status', 
    'Gender', 
    'Education', 
    'Occupation', 
    'Home Owner', 
    'Commute Distance',
    'Region'
  ]

  transformer_pipeline = ColumnTransformer(
    transformers = [
      (
        'num', 
        pipeline_numerical,
        features_numerical
      ),
      (
        'cat', 
        pipeline_categorical,
        features_categorical
      )
    ])
  ```
````

### Task 8: Transformer-Pipeline anwenden

* Die Transformer Pipeline auf dem Trainingsdatenset(X_train) anwenden durch aufrufen der Methode fit_transform() und speichern des Rückgabewert vom Typ Numpy-Array in einer Variable namens 'res'.
* Aus dem Numpy-Array res ein Pandas DataFrame erstellen.


In [13]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 8

  ```{code-block} python
  res = transformer_pipeline.fit_transform(datasets['X_train'])
  pd.DataFrame(res)
  ```
````

### Task 9: Merkmalsbezeichnungen hinzufügen

* Die neuen Feature-Bezeichnungen aus der Transformer Pipeline des Step 'onehot' über die Methode [get_feature_names()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) abfragen und in einer Variable namens feature_categorical_onehot speichern.
* Aus dem Numpy-Array res und den Merkmalsbezeichnungen (feature_catgorical_onehot und features_numerical) ein Pandas DataFrame erstellen und ausgeben.

In [14]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 9

  ```{code-block} python
  feature_categorical_onehot = transformer_pipeline\
    .transformers_[1][1]['onehot']\
    .get_feature_names(features_categorical)

  pd.DataFrame(res, columns=features_numerical+list(feature_categorical_onehot))

  ```
````

### Task 10: Auf alle Datensets anwenden

Wenden Sie die Transformationen auf alle Datensets (Training, Validierung und Test) an und speichern das Ergebnis in den Variablen X_train_transformed, X_val_transformed und X_test_transformed.

In [15]:
# Hier den Code eingeben.

Erstellen Sie aus jedem transformierten Datenset ein Pandas Datenframe inklusive der Spaltenbezeichnungen und speichern diese in den gleichen Variablen. Die neuen Spaltenbezeichungen der kategorischen Daten können sie aus der Variable feature_categorical_onehot des vorherigen Tasks auslesen.

In [16]:
# Hier den Code eingeben.

Geben Sie die ersten Zeilen des Pandas Dataframe X_train_transformed aus:

In [17]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 10

  ```{code-block} python
  # Erste Code-Zelle
  X_train_transformed = transformer_pipeline.fit_transform(datasets['X_train'])
  X_val_transformed = transformer_pipeline.transform(datasets['X_val'])
  X_test_transformed = transformer_pipeline.transform(datasets['X_test'])

  # Zweite Code-Zelle
  X_train_transformed = pd.DataFrame(X_train_transformed, columns=features_numerical+list(feature_categorical_onehot))
  X_val_transformed = pd.DataFrame(X_val_transformed, columns=features_numerical+list(feature_categorical_onehot))
  X_test_transformed = pd.DataFrame(X_test_transformed, columns=features_numerical+list(feature_categorical_onehot))

  # Dritte Code-Zelle
  X_train_transformed.head()
  ```
````

### Task 11: Transformierte Daten speichern

Speichern Sie die transformierten Daten in einem Dictionary unter Verwendung der Keys 
* X_train
* y_train
* X_val
* y_val
* X_test
* y_test  

in einer Variable namens "datasets_tranformed".

In [18]:
# Hier den Code eingeben.

Speichern Sie die transformierten Daten in einer Pickle-Datei unter '../output/bikebuyers/datesets_transformed.pkl'.

In [19]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 11
  
  ```{code-block} python  
  # Erste Code-Zelle
  datasets_transformed = {
      'X_train': X_train_transformed,
      'y_train': datasets['y_train'],
      'X_val': X_val_transformed,
      'y_val': datasets['y_val'],
      'X_test': X_test_transformed,
      'y_test': datasets['y_test']
  }
  
  # Zweite Code-Zelle
  with open('../output/bikebuyers/datesets_transformed.pkl', 'wb') as handle:
    pickle.dump(datasets_transformed, handle)

  ```
````

### Task 12: Pipeline speichern

Speichern Sie die Pipeline mit der besten Parametereinstellung in einer Pickle-Datei.

In [20]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 12

  ```{code-block} python
  with open('../output/bikebuyers/transformer_pipeline.pkl', 'wb') as handle:
    pickle.dump(transformer_pipeline, handle)

  ```
````

### Lösungen Teil 2: Pipelines erstellen, koordinieren und anwenden

In [21]:
# Code Zelle zum testen

````{Dropdown} Lösungen kompakt - Outlier Remover Extended

  ```{code-block} python
  %%writefile transformer_extended.py
  
  import pandas as pd
  import numpy as np
  from sklearn.base import BaseEstimator, TransformerMixin
  
  class OutlierRemoverExtended(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5, strategy='median'):
      self.factor = factor
      self.strategy = strategy

    def fit(self, X, y=None):
      return self

    def transform(self, X, y=None):
      X_ = pd.DataFrame(X)
      q1 = X_.quantile(0.25)
      q3 = X_.quantile(0.75)
      iqr = q3 - q1
      lower_bound = q1 - (self.factor * iqr)
      upper_bound = q3 + (self.factor * iqr)
      X_[((X_ < lower_bound) | (X_ > upper_bound))] = np.nan

      if self.strategy == 'median':
        X_.fillna(X_.median(), inplace=True)
      elif self.strategy == 'mean':
        X_.fillna(X_.mean(), inplace=True)
      else:
        raise ValueError('Invalid value for strategy paramter. Valid values are median or mean.')

      return X_.values
  ```
````

In [22]:
# Code Zelle zum testen

````{Dropdown} Lösungen kompakt - Pipelines

  ```{code-block} python
  import pandas as pd
  import numpy as np
  import sklearn as sklearn
  import pickle
  
  from transformer_extended import OutlierRemoverExtended
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.compose import ColumnTransformer
  
  # Datensets einlesen
  with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:      
    datasets = pickle.load(handle)
  
  # Outlier Remover erstellen
  outlier_remover = OutlierRemoverExtended()
  res = outlier_remover.fit_transform(datasets['X_train'])
  pd.DataFrame(res)
  
  # Pipeline für numerische Daten erstellen
  pipeline_numerical = Pipeline(steps=[
    ('outlier_remover', OutlierRemoverExtended()),
    ('scaler', StandardScaler())
  ])
  
  # Pipeline für kategorische Daten erstellen
  pipeline_categorical = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])

  features_numerical = ['Income', 'Age', 'Cars', 'Children']
  features_categorical = [
    'Marital Status', 
    'Gender', 
    'Education', 
    'Occupation', 
    'Home Owner', 
    'Commute Distance',
    'Region'
  ]

  # Pipelines koordinieren
  transformer_pipeline = ColumnTransformer(
    transformers = [
      (
        'num', 
        pipeline_numerical,
        features_numerical
      ),
      (
        'cat', 
        pipeline_categorical,
        features_categorical
      )
    ])
  
  # Transformationen auf alle Datensets anwenden
  X_train_transformed = transformer_pipeline.fit_transform(datasets['X_train'])
  X_val_transformed = transformer_pipeline.transform(datasets['X_val'])
  X_test_transformed = transformer_pipeline.transform(datasets['X_test'])
  
  feature_categorical_onehot = transformer_pipeline\
    .transformers_[1][1]['onehot']\
    .get_feature_names(features_categorical)

  X_train_transformed = pd.DataFrame(X_train_transformed, columns=features_numerical+list(feature_categorical_onehot))
  X_val_transformed = pd.DataFrame(X_val_transformed, columns=features_numerical+list(feature_categorical_onehot))
  X_test_transformed = pd.DataFrame(X_test_transformed, columns=features_numerical+list(feature_categorical_onehot))
  
  # Transformierte Datensets speichern
  datasets_transformed = {
      'X_train': X_train_transformed,
      'y_train': datasets['y_train'],
      'X_val': X_val_transformed,
      'y_val': datasets['y_val'],
      'X_test': X_test_transformed,
      'y_test': datasets['y_test']
  }
  
  with open('../output/bikebuyers/datesets_transformed.pkl', 'wb') as handle:
    pickle.dump(datasets_transformed, handle)
    
  # Pipeline speichern
  with open('../output/bikebuyers/transformer_pipeline.pkl', 'wb') as handle:
    pickle.dump(transformer_pipeline, handle)
  
  # Ergebnis anzeigen
  print('Erste 5 Zeilen des transformierten Trainingsdatenset')
  datasets_transformed['X_train'].head()
  ```
````

In [23]:
%%writefile transformer_extended.py

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class OutlierRemoverExtended(BaseEstimator, TransformerMixin):
  def __init__(self, factor=1.5, strategy='median'):
    self.factor = factor
    self.strategy = strategy

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    X_ = pd.DataFrame(X)
    q1 = X_.quantile(0.25)
    q3 = X_.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (self.factor * iqr)
    upper_bound = q3 + (self.factor * iqr)
    X_[((X_ < lower_bound) | (X_ > upper_bound))] = np.nan

    if self.strategy == 'median':
      X_.fillna(X_.median(), inplace=True)
    elif self.strategy == 'mean':
      X_.fillna(X_.mean(), inplace=True)
    else:
      raise ValueError('Invalid value for strategy paramter. Valid values are median or mean.')

    return X_.values

Overwriting transformer_extended.py


In [24]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import pickle

from transformer_extended import OutlierRemoverExtended
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Datensets einlesen
with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:      
  datasets = pickle.load(handle)

# Outlier Remover erstellen
outlier_remover = OutlierRemoverExtended()
res = outlier_remover.fit_transform(datasets['X_train'])
pd.DataFrame(res)

# Pipeline für numerische Daten erstellen
pipeline_numerical = Pipeline(steps=[
  ('outlier_remover', OutlierRemoverExtended()),
  ('scaler', StandardScaler())
])

# Pipeline für kategorische Daten erstellen
pipeline_categorical = Pipeline(steps=[
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

features_numerical = ['Income', 'Age', 'Cars', 'Children']
features_categorical = [
  'Marital Status', 
  'Gender', 
  'Education', 
  'Occupation', 
  'Home Owner', 
  'Commute Distance',
  'Region'
]

# Pipelines koordinieren
transformer_pipeline = ColumnTransformer(
  transformers = [
    (
      'num', 
      pipeline_numerical,
      features_numerical
    ),
    (
      'cat', 
      pipeline_categorical,
      features_categorical
    )
  ])

# Transformationen auf alle Datensets anwenden
X_train_transformed = transformer_pipeline.fit_transform(datasets['X_train'])
X_val_transformed = transformer_pipeline.transform(datasets['X_val'])
X_test_transformed = transformer_pipeline.transform(datasets['X_test'])

feature_categorical_onehot = transformer_pipeline\
  .transformers_[1][1]['onehot']\
  .get_feature_names(features_categorical)

X_train_transformed = pd.DataFrame(X_train_transformed, columns=features_numerical+list(feature_categorical_onehot))
X_val_transformed = pd.DataFrame(X_val_transformed, columns=features_numerical+list(feature_categorical_onehot))
X_test_transformed = pd.DataFrame(X_test_transformed, columns=features_numerical+list(feature_categorical_onehot))

# Transformierte Datensets speichern
datasets_transformed = {
    'X_train': X_train_transformed,
    'y_train': datasets['y_train'],
    'X_val': X_val_transformed,
    'y_val': datasets['y_val'],
    'X_test': X_test_transformed,
    'y_test': datasets['y_test']
}

with open('../output/bikebuyers/datesets_transformed.pkl', 'wb') as handle:
  pickle.dump(datasets_transformed, handle)
  
# Pipeline speichern
with open('../output/bikebuyers/transformer_pipeline.pkl', 'wb') as handle:
  pickle.dump(transformer_pipeline, handle)

# Ergebnis anzeigen
print('Erste 5 Zeilen des transformierten Trainingsdatenset')
datasets_transformed['X_train'].head()

Erste 5 Zeilen des transformierten Trainingsdatenset


,Income,Age,Cars,Children,Marital Status_Married,Marital Status_Single,Gender_Female,Gender_Male,Education_Bachelors,Education_Graduate Degree,...,Home Owner_No,Home Owner_Yes,Commute Distance_0-1 Miles,Commute Distance_1-2 Miles,Commute Distance_10+ Miles,Commute Distance_2-5 Miles,Commute Distance_5-10 Miles,Region_Europe,Region_North America,Region_Pacific
0,0.868704,-0.902227,-0.303318,-1.178359,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.507704,-0.625753,-1.380188,-1.178359,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.524602,1.309566,-0.303318,1.272301,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.163602,-0.902227,-1.380188,-0.565694,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.589215,2.138988,-0.303318,0.046971,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
